# 103 Spark - Movielens

The goal of this lab is to run some analysis on a different dataset, [MovieLens](https://grouplens.org/datasets/movielens/).

- Scala
    - [Spark programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
    - [RDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html)
    - [PairRDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/PairRDDFunctions.html)
- Python
    - [Spark programming guide](https://spark.apache.org/docs/3.5.0/rdd-programming-guide.html)
    - [All RDD APIs](https://spark.apache.org/docs/3.5.0/api/python/reference/api/pyspark.RDD.html)

**Download the dataset** from [here](https://big.csr.unibo.it/downloads/bigdata/ml-dataset.zip), unzip it and put it in the ```datasets/big``` folder.

- ml-movies.csv (<u>movieId</u>:Long, title:String, genres:String) 
    - genres are separated by pipelines  (e.g., "comedy|drama|action")
    - each movie is associated with many ratings

- ml-ratings.csv (<u>userId</u>:Long, <u>movieId</u>:Long, rating:Double, year:Int)
    - each rating is associated with many tags
    - ml-ratings-sample.csv is a small sample of ml-ratings.csv, useful for developing
- ml-tags.csv (<u>userId</u>:Long, <u>movieId</u>:Long, <u>tag</u>:String, year:Int) 

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[4]") \
    .appName("Local Spark") \
    .config('spark.ui.port', '4040') \
    .getOrCreate()
sc = spark.sparkContext

sc

<SparkContext master=local[4] appName=Local Spark>

In [2]:
path_to_datasets = "../../../../datasets/big/"

path_ml_movies = path_to_datasets + "ml-movies.csv"
path_ml_ratings = path_to_datasets + "ml-ratings-sample.csv"
path_ml_tags = path_to_datasets + "ml-tags.csv"

In [3]:
import re
from datetime import datetime
from typing import Optional, Tuple

class MovieLensParser:
    no_genres_listed = "(no genres listed)"
    comma_regex = re.compile(r',(?=(?:[^"]*"[^"]*")*[^"]*$)')
    pipe_regex = re.compile(r'\|(?=(?:[^"]*"[^"]*")*[^"]*$)')
    quotes = '"'

    @staticmethod
    def year_from_timestamp(timestamp: str) -> int:
        """Convert from timestamp (string) to year (int)."""
        ts = int(timestamp.strip())
        return datetime.utcfromtimestamp(ts).year

    @staticmethod
    def parse_movie_line(line: str) -> Optional[Tuple[int, str, str]]:
        """Parse a movie record into (movieId, title, genres)."""
        try:
            parts = MovieLensParser.comma_regex.split(line)
            title = parts[1].strip()
            if title.startswith(MovieLensParser.quotes):
                title = title[1:]
            if title.endswith(MovieLensParser.quotes):
                title = title[:-1]
            return (int(parts[0].strip()), title, parts[2].strip())
        except Exception:
            return None

    @staticmethod
    def parse_rating_line(line: str) -> Optional[Tuple[int, int, float, int]]:
        """Parse a rating record into (userId, movieId, rating, year)."""
        try:
            parts = MovieLensParser.comma_regex.split(line)
            year = MovieLensParser.year_from_timestamp(parts[3])
            return int(parts[0].strip()), int(parts[1].strip()), float(parts[2].strip()), year
        except Exception:
            return None

    @staticmethod
    def parse_tag_line(line: str) -> Optional[Tuple[int, int, str, int]]:
        """Parse a tag record into (userId, movieId, tag, year)."""
        try:
            parts = MovieLensParser.comma_regex.split(line)
            year = MovieLensParser.year_from_timestamp(parts[3])
            return (int(parts[0].strip()), int(parts[1].strip()), parts[2].strip(), year)
        except Exception:
            return None


In [4]:
rddMovies = sc.textFile(path_ml_movies).map(MovieLensParser.parse_movie_line).filter(lambda x: x is not None)
rddRatings = sc.textFile(path_ml_ratings).map(MovieLensParser.parse_rating_line).filter(lambda x: x is not None)
rddTags = sc.textFile(path_ml_tags).map(MovieLensParser.parse_tag_line).filter(lambda x: x is not None)

## 103-1 Datasets exploration

Cache the datasets and answer the following questions:

- How many (distinct) users, movies, ratings, and tags?
- How many (distinct) genres?
- On average, how many ratings per user?
- On average, how many ratings per movie?
- On average, how many genres per movie?
- What is the range of ratings?
- Which rating years? (print an ordered list)
- On average, how many ratings per year?

In [5]:
rddMoviesCached = rddMovies.cache()
rddRatingsCached = rddRatings.cache()
rddTagsCached = rddTags.cache()

distinctMovies=rddMoviesCached.map(lambda x: x[0]).distinct().count() #Number of distinct movies
print("Distinct movie: "+str(distinctMovies))
distinctUsers=rddRatingsCached.map(lambda x: x[0]).distinct().count() #Number of distinct users
print("Distinct users: "+str(distinctUsers))
distinctReviewsMultiple=rddRatingsCached.map(lambda x: x[0]+x[1]+x[3]).distinct().count()
distinctReviews=rddRatingsCached.map(lambda x: (x[0]+x[1], 1)).distinct().count()
print("Distinct reviews: "+str(distinctReviews)+", allowing multiples: "+str(distinctReviewsMultiple))
distinctTags=rddTagsCached.map(lambda x: (x[2], 1)).distinct().count()
print("Distinct tags: "+str(distinctTags))

distinctGenres=rddMoviesCached.flatMap(lambda x: x[2].split("|")).distinct().count()
print("Distinct genres: "+str(distinctGenres))

#Average rating per user
rddRatingsCached.map(lambda x: (x[0], (x[2], 1))).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0]/x[1][1])).take(5)

#Average rating per movie
rddRatingsCached.map(lambda x: (x[1], (x[2], 1))).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0]/x[1][1])).take(5)

#Average genres per movie
avgGenres=rddMoviesCached.flatMap(lambda x: x[2].split("|")).count()/distinctMovies
print("Average genres per movie: "+str(avgGenres))

#Range of ratings
min=rddRatingsCached.map(lambda x: x[2]).min()
max=rddRatingsCached.map(lambda x: x[2]).max()
print("Min "+str(min)+" and max rating "+str(max))

#Rating years
years=rddRatingsCached.map(lambda x:x[3]).distinct().sortBy(lambda x: x).collect()

#Ratings per year
print("Ratings per year "+str(distinctReviewsMultiple/len(years)))

Distinct movie: 58098
Distinct users: 10073
Distinct reviews: 123337, allowing multiples: 122695
Distinct tags: 74702
Distinct genres: 20
Average genres per movie: 1.8263451409687081
Min 0.5 and max rating 5.0
Ratings per year 5334.565217391304


## 103-2 Compute the average rating for each movie

- Export the result to a file
- Do not start from cached RDDs
- Evaluate:
  - Join-and-Aggregate
  - Aggregate-and-Join
  - Aggregate-and-BroadcastJoin

In [6]:
path_output_avgRatPerMovie = "../../../../output/avgRatPerMovie"
# rdd.coalesce(1).toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

for (id, rdd) in sc._jsc.getPersistentRDDs().items():         
    rdd.unpersist()

### Join-and-Aggregate

In [16]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[1]))
avgRatPerMovie = rddRatings.\
    map(lambda x: (x[1],x[2])).\
    join(rddMoviesKV).\
    map(lambda x: ((x[0],x[1][1]),(x[1][0],1)) ).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    map(lambda x: (x[0][0], x[0][1], x[1][0]/x[1][1], x[1][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

### Aggregate-and-Join

In [17]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[1]))
avgRatPerMovie = rddRatings.\
    map(lambda x: (x[1],(x[2],1))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    mapValues(lambda x: (x[0]/x[1], x[1])).\
    join(rddMoviesKV).\
    map(lambda x: (x[0], x[1][1], x[1][0][0], x[1][0][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

### Aggregate-and-BroadcastJoin

In [18]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[1]))
bRddMovies = sc.broadcast(rddMoviesKV.collectAsMap())
avgRatPerMovie = rddRatings.\
    map(lambda x: (x[1],(x[2],1))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    mapValues(lambda x: (x[0]/x[1], x[1])).\
    map(lambda x: (x[0],bRddMovies.value.get(x[0]),x[1][0],x[1][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

## 103-3 Compute the average rating for each genre

Two possible workflows:

1. Pre-aggregation (3 shuffles)

  - Aggregate ratings by movieId
  - Join with movies and map to genres
  - Aggregate by genres
  
2. Join & aggregate (2 shuffles)

  - Join with movies and map to genres
  - Aggregate by genres

In [7]:
path_output_avgRatPerMovie = "../../../../output/avgRatPerGenre"

for (id, rdd) in sc._jsc.getPersistentRDDs().items():         
    rdd.unpersist()

# Metodo con 2 shuffle
rddMoviesGenres=rddMovies.flatMap(lambda x: [(x[0], genre) for genre in x[2].split("|")]) #(movie id, genre)
rddRatings.map(lambda x: (x[1], x[2])).join(rddMoviesGenres).map(lambda x: (x[1][1], (x[1][0], 1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0]/x[1][1])).sortByKey().collect()
#(movie id, rating)->joined->(movie id, (genre, rating))->(genre,(rating, 1))

[('(no genres listed)', 3.2050147492625367),
 ('Action', 3.455352917746147),
 ('Adventure', 3.5102944364945143),
 ('Animation', 3.581918830473744),
 ('Children', 3.417999182386264),
 ('Comedy', 3.412333049956375),
 ('Crime', 3.670028609274728),
 ('Documentary', 3.6519604052277472),
 ('Drama', 3.666335509188491),
 ('Fantasy', 3.507156227888689),
 ('Film-Noir', 3.909257375381485),
 ('Horror', 3.2769895869582677),
 ('IMAX', 3.635422788950502),
 ('Musical', 3.5266599723860925),
 ('Mystery', 3.6618355356062136),
 ('Romance', 3.537282367016482),
 ('Sci-Fi', 3.469121237733014),
 ('Thriller', 3.5139459591764077),
 ('War', 3.783042491587761),
 ('Western', 3.561060510887772)]

In [18]:
#Metodo con 3 shuffle, prima media dei punteggi dei film, poi join con i film e generi e faccio la media dei punteggi in base ai film
movieRating=rddRatings.map(lambda x: (x[1], (x[2], 1))).reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1])) #(movie id, sum rating, count rating)
rddMovies.flatMap(lambda x: [(x[0], genre) for genre in x[2].split("|")]).join(movieRating).map(lambda x: (x[1][0], (x[1][1][0], x[1][1][1]))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0]/x[1][1])).sortByKey().collect()
#(movie id, genre)->(movie id, (genre, avg film))->(genre(sum film, num film))

[('(no genres listed)', 3.2050147492625367),
 ('Action', 3.455352917746147),
 ('Adventure', 3.5102944364945143),
 ('Animation', 3.581918830473744),
 ('Children', 3.417999182386264),
 ('Comedy', 3.412333049956375),
 ('Crime', 3.670028609274728),
 ('Documentary', 3.6519604052277472),
 ('Drama', 3.666335509188491),
 ('Fantasy', 3.507156227888689),
 ('Film-Noir', 3.909257375381485),
 ('Horror', 3.2769895869582677),
 ('IMAX', 3.635422788950502),
 ('Musical', 3.5266599723860925),
 ('Mystery', 3.6618355356062136),
 ('Romance', 3.537282367016482),
 ('Sci-Fi', 3.469121237733014),
 ('Thriller', 3.5139459591764077),
 ('War', 3.783042491587761),
 ('Western', 3.561060510887772)]